# Mustererkennung/Machine Learning - Assignment 6 Solution

##### Questions to julian.stastny@fu-berlin.de

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from Classifier_base import Classifier
from math import floor
from LinearRegression import OneVsOne, LeastSquares

In [3]:
data = np.array(pd.read_csv('Data/spambase.data', header=None))

X = data[:,:-1] # features
y = data[:,-1] # Last column is label

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True, stratify=y)

In [4]:
# We binarize the features by asking whether they are above average

# We take the averages of the positive class and the negative class of the training set
averages_pos = np.mean(X_train[y_train==1], axis=0)
averages_neg = np.mean(X_train[y_train==0], axis=0)

average_of_averages = (averages_pos + averages_neg)/2 
# Due to class imbalance, this is not the same as just taking the average of the full training set

X_train_avg = X_train > average_of_averages
X_test_avg = X_test > average_of_averages

In [5]:
class DecisionTree(Classifier):

    def fit(self, X, y, max_depth):
        self.num_features = len(X.T)
        self.size_tree = 2**max_depth - 1
        self.size_array_for_tree = 2**(max_depth+1) - 1
        self.feature_tree = np.ones((self.size_array_for_tree), dtype=int) * (-1)
        self.prob_tree = np.ones((self.size_array_for_tree*2)) * (-1)
        self.split_nodes(X, y, 0)
        return self.feature_tree, self.prob_tree
    
    def predict(self, feature_tree, prob_tree, X):
        predictions = []
        for x in X:
            array_position = 0
            node = feature_tree[array_position]
            while feature_tree[self.left(array_position)] != -1 or feature_tree[self.right(array_position)] != -1:
                if x[node]:
                    array_position = self.right(array_position)
                else:
                    array_position = self.left(array_position)
                prediction = prob_tree[array_position]
                node = feature_tree[array_position]
            predictions += [prediction]
        return predictions
    
    def split_nodes(self, X, y, node_id):
        
        if node_id >= self.size_tree: # Abort if parent node is a leaf
            return
        
        expected_entropies = np.ones(self.num_features) * np.inf # initialize to inf to find true min later
        probs_left = np.empty(self.num_features)
        probs_right = np.empty(self.num_features)
        
        for i, feature in enumerate(X.T):
            if np.sum(X[feature]) == 0 or np.sum(X[np.invert(feature)]) == 0:
                # If one child would get all data points, we don't want to split that way
                continue 
            e_h, prob_left, prob_right = self.weighted_children_entropy(feature, y)
            expected_entropies[i] = e_h
            probs_left[i] = prob_left
            probs_right[i] = prob_right
            
        min_e_h = np.argmin(expected_entropies)
        
        right = X[:,min_e_h]
        left = np.invert(right)
        
        self.feature_tree[node_id] = min_e_h
        self.prob_tree[self.left(node_id)] = probs_left[min_e_h]
        self.prob_tree[self.right(node_id)] = probs_right[min_e_h]
        
        if len(y[right]) == 0 or len(y[left]) == 0:
            return
        # recursive calls
        self.split_nodes(X[left], y[left], self.left(node_id))
        self.split_nodes(X[right], y[right], self.right(node_id))
        
    def entropy(self, p):
        if p == 1 or p == 0: 
            # The entropy is zero if one event is certain
            return 0
        return - (p * np.log(p) + (1-p) * np.log((1-p)))
    
    def weighted_children_entropy(self, feature, y):
        num_datapoints = len(feature)
        weight_right = (feature == True).sum()/num_datapoints
        weight_left = 1 - weight_right
        p = np.sum(y[feature])/len(y[feature]) 
        q = np.sum(y[np.invert(feature)])/len(y[np.invert(feature)])
        entropy_right = weight_right * self.entropy(p)
        entropy_left = weight_left * self.entropy(q)
        return entropy_right + entropy_left, q, p
    
    def left(self, node_id):
        return 2 * node_id + 1
    
    def right(self, node_id):
        return 2 * node_id + 2
    
    def parent(self, node_id):
        return floor((node_id - 1)/2)

In [6]:
decision_tree = DecisionTree()

In [13]:
feature_tree, prob_tree = decision_tree.fit(X_train_avg, y_train, 7)

In [14]:
predictions = decision_tree.predict(feature_tree, prob_tree, X_test_avg)
estimates = (np.array(predictions) > 0.5)

In [15]:
decision_tree.accuracy(y_test, estimates)

0.9052997393570807

In [10]:
decision_tree.confusion_matrix(y_test, estimates).astype(int)

array([[668,  29],
       [ 80, 374]])

In [11]:
class RandomForest(Classifier):
    
    def __init__(self):
        decision_tree = DecisionTree()
    
    def fit(self, X, y, num_trees, num_samples_per_tree, depth):
        self.X = X
        self.y = y
        num_samples = len(X)
        num_features = len(X.T)
        feature_trees = []
        prob_trees = []
        features = []
        for i in range(num_trees):
            random_samples = np.random.randint(0, high=num_samples, size=num_samples_per_tree)
            X = self.X[random_samples]
            y = self.y[random_samples]
            random_features = np.random.randint(0, high=num_features, size=depth*2)
            X = X[:,random_features]
            feature_tree, prob_tree = decision_tree.fit(X, y, depth)
            feature_trees += [feature_tree]
            prob_trees += [prob_tree]
            features += [random_features]
        return feature_trees, prob_trees, features
    
    def predict(self, feature_trees, prob_trees, X):
        predictions = np.empty((len(feature_trees), len(X)))
        for i, (feature_tree, prob_tree, feature_set) in enumerate(zip(feature_trees, prob_trees, features)):
            predictions[i] = decision_tree.predict(feature_tree, prob_tree, X[:,feature_set])
        averaged_predictions = np.mean(predictions, axis=0)
        return averaged_predictions

In [12]:
random_forest = RandomForest()
feature_trees, prob_trees, features = random_forest.fit(X_train_avg, y_train, 50, 1000, 8)

In [13]:
predictions = random_forest.predict(feature_trees, prob_trees, X_test_avg)
estimates = (np.array(predictions) > 0.5)

In [14]:
random_forest.accuracy(y_test, estimates)

0.9209383145091226

In [15]:
random_forest.confusion_matrix(y_test, estimates)

array([[684.,  13.],
       [ 78., 376.]])